In [93]:
import requests
import pandas as pd
from bs4 import BeautifulSoup # to parse census data
import re # regex
import geopandas as gpd
import geopy.distance #to calculate distance between two coordinates
from tqdm import tqdm

In [10]:
coords_1 = (52.2296756, 21.0122287)
coords_2 = (52.406374, 16.9251681)

print(geopy.distance.geodesic(coords_1, coords_2).miles)

173.5818455248231


In [96]:
county_address_filenames = requests.get('https://raw.githubusercontent.com/uva-bi-sdad/national_address_database/main/data/state_county.json').json()
len(county_address_filenames)

1585

In [17]:
county_check_url = 'https://www2.census.gov/geo/maps/DC2020/DC20BLK/st51_va/county/'
r = requests.get(county_check_url)
print(r)

<Response [200]>


In [21]:
soup = BeautifulSoup(r.content)

In [29]:
va_county_links = []
for link in soup.findAll('a'):
    url = link.get('href')
    if isinstance(url, str) and re.search('c\d{5}_', url): 
        va_county_links.append(url)
print(va_county_links)
print(len(va_county_links))

['c51001_accomack/', 'c51003_albemarle/', 'c51005_alleghany/', 'c51007_amelia/', 'c51009_amherst/', 'c51011_appomattox/', 'c51013_arlington/', 'c51015_augusta/', 'c51017_bath/', 'c51019_bedford/', 'c51021_bland/', 'c51023_botetourt/', 'c51025_brunswick/', 'c51027_buchanan/', 'c51029_buckingham/', 'c51031_campbell/', 'c51033_caroline/', 'c51035_carroll/', 'c51036_charles_city/', 'c51037_charlotte/', 'c51041_chesterfield/', 'c51043_clarke/', 'c51045_craig/', 'c51047_culpeper/', 'c51049_cumberland/', 'c51051_dickenson/', 'c51053_dinwiddie/', 'c51057_essex/', 'c51059_fairfax/', 'c51061_fauquier/', 'c51063_floyd/', 'c51065_fluvanna/', 'c51067_franklin/', 'c51069_frederick/', 'c51071_giles/', 'c51073_gloucester/', 'c51075_goochland/', 'c51077_grayson/', 'c51079_greene/', 'c51081_greensville/', 'c51083_halifax/', 'c51085_hanover/', 'c51087_henrico/', 'c51089_henry/', 'c51091_highland/', 'c51093_isle_of_wight/', 'c51095_james_city/', 'c51097_king_and_queen/', 'c51099_king_george/', 'c51101_kin

In [32]:
test_county = county_check_url+va_county_links[0]
print(test_county)
r = requests.get(county_check_url+va_county_links[0])

https://www2.census.gov/geo/maps/DC2020/DC20BLK/st51_va/county/c51001_accomack/


In [33]:
soup = BeautifulSoup(r.content)

In [35]:
for link in soup.findAll('a'):
    url = link.get('href')    
    if isinstance(url, str) and re.search('.txt', url): 
        print(url)
        

DC20BLK_C51001_BLK2MS.txt


# Get blocks per county

In [5]:
url = 'https://www2.census.gov/geo/maps/DC2020/DC20BLK/st51_va/county/c51001_accomack/DC20BLK_C51001_BLK2MS.txt'
df = pd.read_csv(url,sep=';')
df

,TYPE,FULLCODE,STATE,COUNTY,TRACT,BLOCK,PLACE,COUSUB,CONCITY,AIANNH,SHEETS
0,Block,510010901011000,51,1,901.01,1000,16512.0,91196,NaN,NaN,"3,10"
1,Block,510010901011001,51,1,901.01,1001,16512.0,91196,NaN,NaN,10
2,Block,510010901011002,51,1,901.01,1002,16512.0,91196,NaN,NaN,10
3,Block,510010901011003,51,1,901.01,1003,16512.0,91196,NaN,NaN,10
4,Block,510010901011004,51,1,901.01,1004,16512.0,91196,NaN,NaN,10
...,...,...,...,...,...,...,...,...,...,...,...
2178,Block,510019902000008,51,1,9902.00,8,NaN,92096,NaN,NaN,16
2179,Block,510019902000010,51,1,9902.00,10,NaN,92246,NaN,NaN,"16,20"
2180,Block,510019902000011,51,1,9902.00,11,NaN,92246,NaN,NaN,16
2181,Block,510019902000012,51,1,9902.00,12,NaN,92396,NaN,NaN,20


In [6]:
df['BLOCK'].unique()

array([1000, 1001, 1002, 1003, 1004, 1005, 1006, 1007, 1008, 1009, 1010,
       1011, 1012, 1013, 1014, 1015, 1016, 1017, 1018, 1019, 1020, 1021,
       1022, 1023, 1024, 1025, 1026, 1027, 1028, 1029, 1030, 1031, 1032,
       1033, 1034, 1035, 1036, 1037, 1038, 1039, 1040, 1041, 1042, 1043,
       1044, 1045, 1046, 1047, 1048, 1049, 1050, 1051, 1052, 1053, 1054,
       1055, 1056, 1057, 1058, 1059, 1060, 1061, 1062, 1063, 1064, 2000,
       2001, 2002, 2003, 2004, 2005, 2006, 2007, 2008, 2009, 2010, 2011,
       2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019, 2020, 2021, 2022,
       2023, 2024, 2025, 2026, 2027, 2028, 2029, 2030, 2031, 2032, 2033,
       2034, 2035, 2036, 2037, 2038, 2039, 2040, 2041, 2042, 2043, 2044,
       2045, 2046, 2047, 2048, 2049, 2050, 2051, 2052, 2053, 2054, 2055,
       2056, 2057, 2058, 2059, 2060, 2061, 2062, 2063, 2064, 2065, 2066,
       2067, 2068, 2069, 2070, 2071, 2072, 2073, 2074, 2075, 2076, 2077,
       2078, 2079, 2080, 2081, 2082, 2083, 2084, 20

# Get block centroid lat lon

In [ ]:
# Download the TIGER files for blocks (i.e., https://www2.census.gov/geo/tiger/TIGER2020/TABBLOCK20/)

In [67]:
# Set filepath
fp = "../data/tl_2020_51_tabblock20/tl_2020_51_tabblock20.shp"

# Read file using gpd.read_file()
block_df = gpd.read_file(fp)

In [68]:
block_df.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 163491 entries, 0 to 163490
Data columns (total 18 columns):
 #   Column      Non-Null Count   Dtype   
---  ------      --------------   -----   
 0   STATEFP20   163491 non-null  object  
 1   COUNTYFP20  163491 non-null  object  
 2   TRACTCE20   163491 non-null  object  
 3   BLOCKCE20   163491 non-null  object  
 4   GEOID20     163491 non-null  object  
 5   NAME20      163491 non-null  object  
 6   MTFCC20     163491 non-null  object  
 7   UR20        163491 non-null  object  
 8   UACE20      83680 non-null   object  
 9   UATYPE20    83680 non-null   object  
 10  FUNCSTAT20  163491 non-null  object  
 11  ALAND20     163491 non-null  int64   
 12  AWATER20    163491 non-null  int64   
 13  INTPTLAT20  163491 non-null  object  
 14  INTPTLON20  163491 non-null  object  
 15  HOUSING20   163491 non-null  int64   
 16  POP20       163491 non-null  int64   
 17  geometry    163491 non-null  geometry
dtypes: geometry(1), 

In [69]:
block_df['centroid'] = block_df['geometry'].centroid
block_df

/var/folders/31/_flrh_251y578sg72lx1yb780000gn/T/ipykernel_11745/1183721723.py:1: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  block_df['centroid'] = block_df['geometry'].centroid


,STATEFP20,COUNTYFP20,TRACTCE20,BLOCKCE20,GEOID20,NAME20,MTFCC20,UR20,UACE20,UATYPE20,FUNCSTAT20,ALAND20,AWATER20,INTPTLAT20,INTPTLON20,HOUSING20,POP20,geometry,centroid
0,51,065,020101,1027,510650201011027,Block 1027,G5040,R,NaN,NaN,S,1547604,0,+37.8704905,-078.2665116,18,40,"POLYGON ((-78.27375 37.87051, -78.27324 37.870...",POINT (-78.26651 37.87049)
1,51,197,050101,1061,511970501011061,Block 1061,G5040,U,97453,U,S,15055,0,+36.9409610,-081.0882937,6,8,"POLYGON ((-81.08860 36.94152, -81.08830 36.941...",POINT (-81.08829 36.94096)
2,51,197,050101,3063,511970501013063,Block 3063,G5040,R,NaN,NaN,S,11611,0,+36.9412795,-080.9753199,0,0,"POLYGON ((-80.97608 36.94150, -80.97607 36.941...",POINT (-80.97532 36.94128)
3,51,197,050301,1036,511970503011036,Block 1036,G5040,R,NaN,NaN,S,166245,0,+36.8857859,-081.2774449,3,4,"POLYGON ((-81.28102 36.88490, -81.28071 36.885...",POINT (-81.27744 36.88579)
4,51,199,050500,2052,511990505002052,Block 2052,G5040,R,NaN,NaN,S,292731,0,+37.2259510,-076.5020127,0,0,"POLYGON ((-76.50644 37.22507, -76.50640 37.225...",POINT (-76.50201 37.22595)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
163486,51,159,040100,3037,511590401003037,Block 3037,G5040,R,NaN,NaN,S,4561610,48673,+37.8986088,-076.7515189,18,38,"POLYGON ((-76.76612 37.88520, -76.76576 37.885...",POINT (-76.74973 37.89810)
163487,51,109,950500,3014,511099505003014,Block 3014,G5040,R,NaN,NaN,S,43464,0,+37.8490036,-077.7906355,3,19,"POLYGON ((-77.79179 37.84769, -77.79164 37.848...",POINT (-77.79064 37.84900)
163488,51,109,950102,1008,511099501021008,Block 1008,G5040,R,NaN,NaN,S,0,4267,+38.0630940,-077.8308694,0,0,"POLYGON ((-77.83180 38.06289, -77.83098 38.063...",POINT (-77.83087 38.06309)
163489,51,650,011800,4009,516500118004009,Block 4009,G5040,U,90892,U,S,81063,0,+37.0129132,-076.3642622,30,188,"POLYGON ((-76.36596 37.01361, -76.36514 37.014...",POINT (-76.36426 37.01291)


In [70]:
block_df

,STATEFP20,COUNTYFP20,TRACTCE20,BLOCKCE20,GEOID20,NAME20,MTFCC20,UR20,UACE20,UATYPE20,FUNCSTAT20,ALAND20,AWATER20,INTPTLAT20,INTPTLON20,HOUSING20,POP20,geometry,centroid
0,51,065,020101,1027,510650201011027,Block 1027,G5040,R,NaN,NaN,S,1547604,0,+37.8704905,-078.2665116,18,40,"POLYGON ((-78.27375 37.87051, -78.27324 37.870...",POINT (-78.26651 37.87049)
1,51,197,050101,1061,511970501011061,Block 1061,G5040,U,97453,U,S,15055,0,+36.9409610,-081.0882937,6,8,"POLYGON ((-81.08860 36.94152, -81.08830 36.941...",POINT (-81.08829 36.94096)
2,51,197,050101,3063,511970501013063,Block 3063,G5040,R,NaN,NaN,S,11611,0,+36.9412795,-080.9753199,0,0,"POLYGON ((-80.97608 36.94150, -80.97607 36.941...",POINT (-80.97532 36.94128)
3,51,197,050301,1036,511970503011036,Block 1036,G5040,R,NaN,NaN,S,166245,0,+36.8857859,-081.2774449,3,4,"POLYGON ((-81.28102 36.88490, -81.28071 36.885...",POINT (-81.27744 36.88579)
4,51,199,050500,2052,511990505002052,Block 2052,G5040,R,NaN,NaN,S,292731,0,+37.2259510,-076.5020127,0,0,"POLYGON ((-76.50644 37.22507, -76.50640 37.225...",POINT (-76.50201 37.22595)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
163486,51,159,040100,3037,511590401003037,Block 3037,G5040,R,NaN,NaN,S,4561610,48673,+37.8986088,-076.7515189,18,38,"POLYGON ((-76.76612 37.88520, -76.76576 37.885...",POINT (-76.74973 37.89810)
163487,51,109,950500,3014,511099505003014,Block 3014,G5040,R,NaN,NaN,S,43464,0,+37.8490036,-077.7906355,3,19,"POLYGON ((-77.79179 37.84769, -77.79164 37.848...",POINT (-77.79064 37.84900)
163488,51,109,950102,1008,511099501021008,Block 1008,G5040,R,NaN,NaN,S,0,4267,+38.0630940,-077.8308694,0,0,"POLYGON ((-77.83180 38.06289, -77.83098 38.063...",POINT (-77.83087 38.06309)
163489,51,650,011800,4009,516500118004009,Block 4009,G5040,U,90892,U,S,81063,0,+37.0129132,-076.3642622,30,188,"POLYGON ((-76.36596 37.01361, -76.36514 37.014...",POINT (-76.36426 37.01291)


In [74]:
block_df['id'] = block_df['STATEFP20'] + block_df['COUNTYFP20']
block_df

,STATEFP20,COUNTYFP20,TRACTCE20,BLOCKCE20,GEOID20,NAME20,MTFCC20,UR20,UACE20,UATYPE20,FUNCSTAT20,ALAND20,AWATER20,INTPTLAT20,INTPTLON20,HOUSING20,POP20,geometry,centroid,id
0,51,065,020101,1027,510650201011027,Block 1027,G5040,R,NaN,NaN,S,1547604,0,+37.8704905,-078.2665116,18,40,"POLYGON ((-78.27375 37.87051, -78.27324 37.870...",POINT (-78.26651 37.87049),51065
1,51,197,050101,1061,511970501011061,Block 1061,G5040,U,97453,U,S,15055,0,+36.9409610,-081.0882937,6,8,"POLYGON ((-81.08860 36.94152, -81.08830 36.941...",POINT (-81.08829 36.94096),51197
2,51,197,050101,3063,511970501013063,Block 3063,G5040,R,NaN,NaN,S,11611,0,+36.9412795,-080.9753199,0,0,"POLYGON ((-80.97608 36.94150, -80.97607 36.941...",POINT (-80.97532 36.94128),51197
3,51,197,050301,1036,511970503011036,Block 1036,G5040,R,NaN,NaN,S,166245,0,+36.8857859,-081.2774449,3,4,"POLYGON ((-81.28102 36.88490, -81.28071 36.885...",POINT (-81.27744 36.88579),51197
4,51,199,050500,2052,511990505002052,Block 2052,G5040,R,NaN,NaN,S,292731,0,+37.2259510,-076.5020127,0,0,"POLYGON ((-76.50644 37.22507, -76.50640 37.225...",POINT (-76.50201 37.22595),51199
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
163486,51,159,040100,3037,511590401003037,Block 3037,G5040,R,NaN,NaN,S,4561610,48673,+37.8986088,-076.7515189,18,38,"POLYGON ((-76.76612 37.88520, -76.76576 37.885...",POINT (-76.74973 37.89810),51159
163487,51,109,950500,3014,511099505003014,Block 3014,G5040,R,NaN,NaN,S,43464,0,+37.8490036,-077.7906355,3,19,"POLYGON ((-77.79179 37.84769, -77.79164 37.848...",POINT (-77.79064 37.84900),51109
163488,51,109,950102,1008,511099501021008,Block 1008,G5040,R,NaN,NaN,S,0,4267,+38.0630940,-077.8308694,0,0,"POLYGON ((-77.83180 38.06289, -77.83098 38.063...",POINT (-77.83087 38.06309),51109
163489,51,650,011800,4009,516500118004009,Block 4009,G5040,U,90892,U,S,81063,0,+37.0129132,-076.3642622,30,188,"POLYGON ((-76.36596 37.01361, -76.36514 37.014...",POINT (-76.36426 37.01291),51650


In [75]:
len(block_df['id'].unique())

133

# Get address per block

In [90]:
fdf = block_df[['centroid', 'id', 'GEOID20']]
fdf = fdf.rename(columns = {'id': 'county_id', 'GEOID20': 'geoid20'})
fdf.sort_values('county_id')
fdf

,centroid,county_id,geoid20
0,POINT (-78.26651 37.87049),51065,510650201011027
1,POINT (-81.08829 36.94096),51197,511970501011061
2,POINT (-80.97532 36.94128),51197,511970501013063
3,POINT (-81.27744 36.88579),51197,511970503011036
4,POINT (-76.50201 37.22595),51199,511990505002052
...,...,...,...
163486,POINT (-76.74973 37.89810),51159,511590401003037
163487,POINT (-77.79064 37.84900),51109,511099505003014
163488,POINT (-77.83087 38.06309),51109,511099501021008
163489,POINT (-76.36426 37.01291),51650,516500118004009


In [137]:
fdf['centroid'] = fdf['centroid'].apply(lambda x: (x.x, x.y))
fdf

,centroid,county_id,geoid20,lon,lat
0,"(-78.26651157648517, 37.87049080769003)",51065,510650201011027,-78.266512,37.870491
1,"(-81.0882937021228, 36.940961018504034)",51197,511970501011061,-81.088294,36.940961
2,"(-80.97531992101706, 36.9412794596366)",51197,511970501013063,-80.975320,36.941279
3,"(-81.27744485470693, 36.88578586696921)",51197,511970503011036,-81.277445,36.885786
4,"(-76.5020127409249, 37.22595094303625)",51199,511990505002052,-76.502013,37.225951
...,...,...,...,...,...
163486,"(-76.74973475094208, 37.89809558995467)",51159,511590401003037,-76.749735,37.898096
163487,"(-77.79063544221452, 37.84900361722527)",51109,511099505003014,-77.790635,37.849004
163488,"(-77.83086939995073, 38.063094012482374)",51109,511099501021008,-77.830869,38.063094
163489,"(-76.36426220239646, 37.012913187910506)",51650,516500118004009,-76.364262,37.012913


In [139]:
fdf

,centroid,county_id,geoid20
0,"(-78.26651157648517, 37.87049080769003)",51065,510650201011027
1,"(-81.0882937021228, 36.940961018504034)",51197,511970501011061
2,"(-80.97531992101706, 36.9412794596366)",51197,511970501013063
3,"(-81.27744485470693, 36.88578586696921)",51197,511970503011036
4,"(-76.5020127409249, 37.22595094303625)",51199,511990505002052
...,...,...,...
163486,"(-76.74973475094208, 37.89809558995467)",51159,511590401003037
163487,"(-77.79063544221452, 37.84900361722527)",51109,511099505003014
163488,"(-77.83086939995073, 38.063094012482374)",51109,511099501021008
163489,"(-76.36426220239646, 37.012913187910506)",51650,516500118004009


In [132]:
# coords_1 = fdf['coor'][0]
# coords_2 = (52.406374, 16.9251681)

# print(geopy.distance.geodesic(coords_1, coords_2).miles)

9053.21183038715


# Single county testing

In [145]:
tdf = fdf[fdf['county_id'] == '51001']
tdf

,centroid,county_id,geoid20
756,"(-75.48690482639833, 37.949252542219384)",51001,510010902012084
918,"(-75.76338486652637, 37.66227930958755)",51001,510010907004037
919,"(-75.5260296753147, 37.92930425083767)",51001,510010902023006
922,"(-75.8463823965764, 37.61790786138303)",51001,510010907005047
1026,"(-75.8124596249652, 37.721198664130085)",51001,510010906004019
...,...,...,...
163282,"(-75.78665572822679, 37.53109437902871)",51001,510010908003047
163283,"(-75.58170971587099, 37.778303660603854)",51001,510010904012095
163284,"(-75.61097511965288, 37.753750649378)",51001,510010904021024
163328,"(-75.62955164622439, 37.65327901798022)",51001,510010904022077


In [149]:
tdf = tdf.sample(100)

# Convert long lat to coordinate, because distance is calculated using a coordinate

In [143]:
cadf = pd.read_csv("https://github.com/uva-bi-sdad/national_address_database/raw/main/data/va_accomack.csv.xz")
cadf['coor'] = cadf.apply(lambda x: (x['longitude'], x['latitude']), axis=1)
cadf

,state,county,zip,longitude,latitude,address,id,coor
0,va,accomack,23420,-75.782732,37.586385,"33282 grant street,painter,va,23420",va_accomack,"(-75.78273194373745, 37.586384624451455)"
1,va,accomack,23480,-75.687351,37.610529,"7 richardson avenue,wachapreague,va,23480",va_accomack,"(-75.68735058936421, 37.610528599630726)"
2,va,accomack,23480,-75.689122,37.610878,"17 richardson avenue,wachapreague,va,23480",va_accomack,"(-75.68912243182051, 37.6108780537332)"
3,va,accomack,23480,-75.687581,37.610198,"40 brooklyn avenue,wachapreague,va,23480",va_accomack,"(-75.68758139979836, 37.61019849733271)"
4,va,accomack,23480,-75.686306,37.610433,"1 richardson avenue,wachapreague,va,23480",va_accomack,"(-75.6863059726563, 37.61043334427158)"
...,...,...,...,...,...,...,...,...
26417,va,accomack,23417,-75.810774,37.709995,"15227 broadway road,onancock,va,23417",va_accomack,"(-75.81077434653461, 37.709995318705616)"
26418,va,accomack,23410,-75.761638,37.653465,"28361 beacon road,melfa,va,23410",va_accomack,"(-75.76163822795358, 37.653464754699584)"
26419,va,accomack,23356,-75.419386,37.989802,"3238 dock court,greenbackville,va,23356",va_accomack,"(-75.41938613419619, 37.98980185736946)"
26420,va,accomack,23356,-75.422857,37.989581,"3373 captains corr,greenbackville,va,23356",va_accomack,"(-75.42285705018088, 37.9895806209073)"


In [164]:
def match_address(block_df, county_address_df):
    cadf = county_address_df
    
    matched_address = []
    distances = []
    matched_coor = []

    descriptor = county_address_df['id'].unique()[0]
    
    pbar = tqdm(block_df['centroid'])
    pbar.set_description('parsing %s' % descriptor)
    for coor in pbar:
        cadf['dist'] = cadf['coor'].apply(lambda x: geopy.distance.geodesic(x, coor).miles)
        # find the minimum value in a data frame
        element = cadf[cadf['dist'] == min(cadf['dist'])]
        matched_address.append(element['address'].values[0])
        matched_coor.append(element['coor'].values[0])
        distances.append(element['dist'].values[0])

    block_df['matched_address'] = matched_address
    tdblock_dff['distance_to_address'] = distances
    block_df['matched_coor'] = matched_coor
    return block_df

In [ ]:
tdf = match_address(tdf, cadf)
tdf

parsing va_fluvanna:   1%|██▏                                                                                                                                                                                                                    | 1/100 [00:00<01:37,  1.01it/s]

In [153]:
tdf['distance_to_address'].describe()

count    100.000000
mean       0.244651
std        0.558287
min        0.001545
25%        0.024691
50%        0.073175
75%        0.187045
max        3.510953
Name: distance_to_address, dtype: float64

In [155]:
tdf[tdf['distance_to_address']> 1]

,centroid,county_id,geoid20,match,matched_address,distance_to_address,matched_coor
143521,"(-75.90927353223844, 37.808131137876856)",51001,510010906001032,"(28438 masons beach road,harborton,va,23389, (...","28438 masons beach road,harborton,va,23389",3.292886,"(-75.8807542866555, 37.65245586428504)"
134009,"(-75.63176676497558, 37.592460211282166)",51001,510010908001042,"(26399 sea breeze drive,onley,va,23418, (-75.6...","26399 sea breeze drive,onley,va,23418",1.422623,"(-75.63079760969158, 37.67498418911909)"
132486,"(-75.68509607046165, 37.456764036480024)",51001,510010908002145,"(22395 marlin lane,wachapreague,va,23480, (-75...","22395 marlin lane,wachapreague,va,23480",2.490132,"(-75.69362165829406, 37.59779515218349)"
158857,"(-75.92392310332694, 37.79148318364156)",51001,510019902000005,"(9491 scarboroughs neck road,davis wharf,va,23...","9491 scarboroughs neck road,davis wharf,va,23345",3.510953,"(-75.91996530144628, 37.58410744730508)"


# For each block, get info from the NAD, then return the closest

In [98]:
va = [c for c in county_address_filenames if c.split('_')[0]=='va']
va

['va_accomack',
 'va_albemarle',
 'va_alexandria_city',
 'va_alleghany',
 'va_amelia',
 'va_amherst',
 'va_appomattox',
 'va_arlington',
 'va_augusta',
 'va_bath',
 'va_bedford',
 'va_bland',
 'va_botetourt',
 'va_bristol_city',
 'va_brunswick',
 'va_buchanan',
 'va_buckingham',
 'va_buena_vista_city',
 'va_campbell',
 'va_caroline',
 'va_carroll',
 'va_charles_city',
 'va_charlotte',
 'va_charlottesville_city',
 'va_chesapeake_city',
 'va_chesterfield',
 'va_clarke',
 'va_colonial_heights_city',
 'va_covington_city',
 'va_craig',
 'va_culpeper',
 'va_cumberland',
 'va_danville_city',
 'va_dickenson',
 'va_dinwiddie',
 'va_emporia_city',
 'va_essex',
 'va_fairfax',
 'va_fairfax_city',
 'va_falls_church_city',
 'va_fauquier',
 'va_floyd',
 'va_fluvanna',
 'va_franklin',
 'va_franklin_city',
 'va_frederick',
 'va_fredericksburg_city',
 'va_galax_city',
 'va_giles',
 'va_gloucester',
 'va_goochland',
 'va_grayson',
 'va_greene',
 'va_greensville',
 'va_halifax',
 'va_hampton_city',
 'va_h

In [99]:
fips_county = pd.read_csv('https://raw.githubusercontent.com/uva-bi-sdad/national_address_database/main/data/fips_county.csv.xz', dtype={'fips': object}) # read fips as a string
fips_county

,fips,county
0,01000,alabama
1,01001,autauga_county
2,01003,baldwin_county
3,01005,barbour_county
4,01007,bibb_county
...,...,...
3190,56037,sweetwater_county
3191,56039,teton_county
3192,56041,uinta_county
3193,56043,washakie_county


In [159]:
for county in fdf['county_id'].unique():
    cdf = fdf[fdf['county_id']==county]
    # now get the name of the file that corresponds with the county name
    county_name = fips_county[fips_county['fips'] == county]['county'].values[0]
    # print(county_name)
    cad = [s for s in va if county_name.replace('_county','') in s]
    if len(cad) >1: # if there can be two things mapped, this because the county is removed, so you can remove the selection with city
        cad = [v for v in cad if not 'city' in v]
   
    assert len(cad) == 1, 'Coud not find: %s' % county_name
    cad = cad[0]
    cadf = pd.read_csv("https://github.com/uva-bi-sdad/national_address_database/raw/main/data/%s.csv.xz" % cad)
    # print(cadf)
    # Now, get the cloest address to the given block address and return it if is close enough (1 mile)
    cadf['coor'] = cadf.apply(lambda x: (x['longitude'], x['latitude']), axis=1)
    cdf = match_address(cdf, cadf)
    cdf.to_csv('../data/%s.csv.xz' % cad, index=False)

  1%|█▉                                                                                                                                                                                                                                          | 5/627 [00:05<11:00,  1.06s/it]


KeyboardInterrupt: 

In [21]:
pdf = pd.read_csv("https://github.com/uva-bi-sdad/national_address_database/raw/main/data/%s.csv.xz" % va[0])
pdf

,state,county,zip,longitude,latitude,address,id
0,va,accomack,23420,-75.782732,37.586385,"33282 grant street,painter,va,23420",va_accomack
1,va,accomack,23480,-75.687351,37.610529,"7 richardson avenue,wachapreague,va,23480",va_accomack
2,va,accomack,23480,-75.689122,37.610878,"17 richardson avenue,wachapreague,va,23480",va_accomack
3,va,accomack,23480,-75.687581,37.610198,"40 brooklyn avenue,wachapreague,va,23480",va_accomack
4,va,accomack,23480,-75.686306,37.610433,"1 richardson avenue,wachapreague,va,23480",va_accomack
...,...,...,...,...,...,...,...
26417,va,accomack,23417,-75.810774,37.709995,"15227 broadway road,onancock,va,23417",va_accomack
26418,va,accomack,23410,-75.761638,37.653465,"28361 beacon road,melfa,va,23410",va_accomack
26419,va,accomack,23356,-75.419386,37.989802,"3238 dock court,greenbackville,va,23356",va_accomack
26420,va,accomack,23356,-75.422857,37.989581,"3373 captains corr,greenbackville,va,23356",va_accomack


# Get county address csv by fips

In [79]:
fip = fips_county[fips_county['county'].str.contains('accomack')]['fips'].values[0]
fip

'51001'

In [32]:
pt = data.iloc[0]['centroid']
print(pt.x)
print(pt.y)

-78.26651157648517
37.87049080769003


In [ ]:
pdf['dist'] = geopy.distance.geodesic(coords_1, coords_2).miles